In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref,Ref.
0,JANUARY,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay; Max Landi...,[2],NaN
1,JANUARY,13.0,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15.0,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15.0,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
171,DECEMBER,22.0,Sing 2,Universal Pictures / Illumination,Garth Jennings (director/screenplay); Matthew ...,NaN,[132]
172,DECEMBER,22.0,The King's Man,20th Century Studios / Marv Films,Matthew Vaughn (director/screenplay); Karl Gaj...,NaN,[90]
173,DECEMBER,22.0,Downton Abbey 2,Focus Features / Carnival Films,Simon Curtis (director); Julian Fellowes (scre...,NaN,[133]
174,DECEMBER,25.0,Cyrano,Metro-Goldwyn-Mayer / Working Title Films,Joe Wright (director); Erica Schmidt (screenpl...,NaN,[134]


In [10]:
df_2021 = df[['Title','Cast and crew']]

In [11]:
df_2021

,Title,Cast and crew
0,Shadow in the Cloud,Roseanne Liang (director/screenplay; Max Landi...
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...
3,The Dig,Simon Stone (director); Moira Buffini (screenp...
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa..."
...,...,...
171,Sing 2,Garth Jennings (director/screenplay); Matthew ...
172,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...
173,Downton Abbey 2,Simon Curtis (director); Julian Fellowes (scre...
174,Cyrano,Joe Wright (director); Erica Schmidt (screenpl...


In [12]:
# !pip install tmdbv3api

In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'a6d0cb3b7c1112716b5ad4788420eade'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [15]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-15-252c1cda1e49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))


In [16]:
df_2021

,Title,Cast and crew,genres
0,Shadow in the Cloud,Roseanne Liang (director/screenplay; Max Landi...,Horror Action Drama War
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Crime Drama
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction
...,...,...,...
171,Sing 2,Garth Jennings (director/screenplay); Matthew ...,Music Animation Comedy Family
172,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...,Action Adventure Comedy
173,Downton Abbey 2,Simon Curtis (director); Julian Fellowes (scre...,Drama
174,Cyrano,Joe Wright (director); Erica Schmidt (screenpl...,Drama Comedy History Romance


In [17]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [18]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-18-059d9cfe3ca6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))


In [19]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [20]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-20-fbf9a4980306>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [21]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [22]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-22-597eaf6de001>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [23]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [24]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [25]:
df_2021

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Roseanne Liang (director/screenplay; Max Landi...,Horror Action Drama War,Roseanne Liang (director/screenplay; Max Landi...,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Crime Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham
...,...,...,...,...,...,...,...
171,Sing 2,Garth Jennings (director/screenplay); Matthew ...,Music Animation Comedy Family,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson
172,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...,Action Adventure Comedy,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans
173,Downton Abbey 2,Simon Curtis (director); Julian Fellowes (scre...,Drama,Simon Curtis,Hugh Bonneville,Jim Carter,Michelle Dockery
174,Cyrano,Joe Wright (director); Erica Schmidt (screenpl...,Drama Comedy History Romance,Joe Wright,Peter Dinklage,Haley Bennett,Ben Mendelsohn


In [26]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [27]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [28]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang (director/screenplay; Max Landi...,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action Drama War,Shadow in the Cloud
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Crime Drama,The White Tiger
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,Locked Down
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,The Dig
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,Outside the Wire
...,...,...,...,...,...,...
171,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Music Animation Comedy Family,Sing 2
172,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Comedy,The King's Man
173,Simon Curtis,Hugh Bonneville,Jim Carter,Michelle Dockery,Drama,Downton Abbey 2
174,Joe Wright,Peter Dinklage,Haley Bennett,Ben Mendelsohn,Drama Comedy History Romance,Cyrano


In [29]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [30]:
new_df21.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     3
actor_3_name     9
genres           1
movie_title      1
comb             9
dtype: int64

In [31]:
new_df21 = new_df21.dropna(how='any')

In [32]:
new_df21.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [33]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

<ipython-input-33-6e765124d575>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df21['movie_title'] = new_df21['movie_title'].str.lower()


In [34]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang (director/screenplay; Max Landi...,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action Drama War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Crime Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,the dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,outside the wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
171,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Music Animation Comedy Family,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
172,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Comedy,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
173,Simon Curtis,Hugh Bonneville,Jim Carter,Michelle Dockery,Drama,downton abbey 2,Hugh Bonneville Jim Carter Michelle Dockery Si...
174,Joe Wright,Peter Dinklage,Haley Bennett,Ben Mendelsohn,Drama Comedy History Romance,cyrano,Peter Dinklage Haley Bennett Ben Mendelsohn Jo...


In [35]:
old_df = pd.read_csv('/mnt/ntfs-d/akallada/Desktop/projects/mrs/datasets/final_data.csv')

In [36]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5867,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
5868,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5869,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action Thriller,1917,George MacKay Dean-Charles Chapman Mark Strong...
5870,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [37]:
final_df = old_df.append(new_df21,ignore_index=True)

In [38]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6034,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Music Animation Comedy Family,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
6035,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Comedy,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6036,Simon Curtis,Hugh Bonneville,Jim Carter,Michelle Dockery,Drama,downton abbey 2,Hugh Bonneville Jim Carter Michelle Dockery Si...
6037,Joe Wright,Peter Dinklage,Haley Bennett,Ben Mendelsohn,Drama Comedy History Romance,cyrano,Peter Dinklage Haley Bennett Ben Mendelsohn Jo...


In [39]:
final_df.to_csv('main_data.csv',index=False)

In [40]:
final_df.shape

(6039, 7)

In [41]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6034,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Music Animation Comedy Family,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
6035,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Comedy,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6036,Simon Curtis,Hugh Bonneville,Jim Carter,Michelle Dockery,Drama,downton abbey 2,Hugh Bonneville Jim Carter Michelle Dockery Si...
6037,Joe Wright,Peter Dinklage,Haley Bennett,Ben Mendelsohn,Drama Comedy History Romance,cyrano,Peter Dinklage Haley Bennett Ben Mendelsohn Jo...
